# Eat Safe, Love

## Notebook Set Up

In [42]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [43]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [44]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [45]:
# review the collections in our database
collection = db["collections"]
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food'), 'collections')

In [46]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [47]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with hygiene score equal to 20: {count}")
# Display the first document in the results using pprint
results = establishments.find(query)
results[0]

Number of establishments with hygiene score equal to 20: 41


{'_id': ObjectId('675663c61c82c4670b196997'),
 'FHRSID': 110681,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': '4029',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'PostCode': 'BN21 1BU',
 'Phone': '',
 'RatingValue': 0,
 'RatingKey': 'fhrs_0_en-gb',
 'RatingDate': '2021-09-23T00:00:00',
 'LocalAuthorityCode': '102',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'scores': {'Hygiene': 20, 'Structural': 20, 'ConfidenceInManagement': 20},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': Decimal128('0.27694'),
  'latitude': Decimal128('50.769705')},
 'RightToReply': '',
 'Distance': 4613.888288172291,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001

In [48]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(result_df)}")
# Display the first 10 rows of the DataFrame
print(result_df.head(10))

Number of rows in the DataFrame: 41
                        _id   FHRSID  ChangesByServerID  \
0  675663c61c82c4670b196997   110681                  0   
1  675663c61c82c4670b196d16   612039                  0   
2  675663c61c82c4670b197022   730933                  0   
3  675663c71c82c4670b197210   172735                  0   
4  675663c71c82c4670b197223   172953                  0   
5  675663c71c82c4670b197bbd   512854                  0   
6  675663c71c82c4670b197ddf  1537089                  0   
7  675663c71c82c4670b19930c   155648                  0   
8  675663c71c82c4670b19974d  1012883                  0   
9  675663c71c82c4670b199f60   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza   
4             PI/000024532   

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [60]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex':'London'}, 'RatingValue': {'$gte': 4}}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with London as Local Authority and RatingValue >= 4: {count}")
# Display the first document in the results using pprint
results2 = establishments.find(query)
for document in results:
    pprint(document)

Number of establishments with London as Local Authority and RatingValue >= 4: 33


In [61]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(results2)
# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(result_df)}")
# Display the first 10 rows of the DataFrame
print(result_df.head(10))

Number of rows in the DataFrame: 33
                        _id   FHRSID  ChangesByServerID  \
0  675663c71c82c4670b1983b7   621707                  0   
1  675663c71c82c4670b1986d9  1130836                  0   
2  675663c71c82c4670b199226   293783                  0   
3  675663c71c82c4670b19a026  1315095                  0   
4  675663c71c82c4670b19a027   294474                  0   
5  675663c71c82c4670b19a730   294900                  0   
6  675663c81c82c4670b19c83d   293756                  0   
7  675663c81c82c4670b19cc04   878523                  0   
8  675663c81c82c4670b19cc1b   293772                  0   
9  675663c81c82c4670b19cc1d   294606                  0   

  LocalAuthorityBusinessID                          BusinessName  \
0             PI/000025307                             Charlie's   
1             PI/000034075               Mv City Cruises Erasmus   
2             PI/000002614             Benfleet Motor Yacht Club   
3             PI/000036464  Coombs Caterin

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [67]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}
}
sort = [("scores.Hygiene", 1)]
limit = 20

results3 = establishments.find(query).sort(sort).limit(limit)

# Print the results
result_list = list(results3)
result_list

[{'_id': ObjectId('675663c81c82c4670b19c212'),
  'FHRSID': 694478,
  'ChangesByServerID': 0,
  'LocalAuthorityBusinessID': 'PI/000086506',
  'BusinessName': 'Atlantic Fish Bar',
  'BusinessType': 'Takeaway/sandwich shop',
  'BusinessTypeID': 7844,
  'AddressLine1': '35 Lakedale Road',
  'AddressLine2': '',
  'AddressLine3': 'Plumstead',
  'AddressLine4': 'Greenwich',
  'PostCode': 'SE18 1PR',
  'Phone': '',
  'RatingValue': 5,
  'RatingKey': 'fhrs_5_en-gb',
  'RatingDate': '2021-06-16T00:00:00',
  'LocalAuthorityCode': '511',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 5},
  'SchemeType': 'FHRS',
  'geocode': {'longitude': Decimal128('0.0912164'),
   'latitude': Decimal128('51.4867296')},
  'RightToReply': '',
  'Distance': 4646.974611855838,
  'NewRatingPending': False,
  'meta': {'dataSource': Non

In [69]:
# Convert result to Pandas DataFrame
result_df = pd.DataFrame(result_list)
print(f"Number of rows in the DataFrame: {len(result_df)}")
print(result_df.head(10))

Number of rows in the DataFrame: 20
                        _id   FHRSID  ChangesByServerID  \
0  675663c81c82c4670b19c212   694478                  0   
1  675663c81c82c4670b19c2de   732579                  0   
2  675663c81c82c4670b19c2fa   694338                  0   
3  675663c81c82c4670b19c297   695400                  0   
4  675663c81c82c4670b19c2ba   815071                  0   
5  675663c81c82c4670b19c25b   940026                  0   
6  675663c81c82c4670b19c25c  1211907                  0   
7  675663c81c82c4670b19c30c  1100050                  0   
8  675663c81c82c4670b19c1da  1380578                  0   
9  675663c81c82c4670b19c214   909912                  0   

  LocalAuthorityBusinessID                         BusinessName  \
0             PI/000086506                    Atlantic Fish Bar   
1             PI/000116525                           The Belfry   
2             PI/000061059                Conway Primary School   
3             PI/000184827                    

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [70]:
# Create a pipeline that:
pipeline = [
# 1. Matches establishments with a hygiene score of 0
 {'$match': {'scores.Hygiene': 0}},
# 2. Groups the matches by Local Authority
{'$group': {'_id': "$LocalAuthorityName", 'count': {'$sum': 1}}},
# 3. Sorts the matches from highest to lowest
 {'$sort': {'count': -1}}
]
results = list(establishments.aggregate(pipeline))
# Print the number of documents in the result
print("Number of Local Authorities in result:", len(results))
# Print the first 10 results
pprint(results[0:10])

Number of Local Authorities in result: 56
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [71]:
# Convert the result to a Pandas DataFrame
result_list = list(results)
result_df = pd.DataFrame(result_list)
# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(result_df)}")
# Display the first 10 rows of the DataFrame
print(result_df.head(10))

Number of rows in the DataFrame: 56
               _id  count
0           Thanet   1130
1        Greenwich    882
2        Maidstone    713
3           Newham    711
4            Swale    686
5       Chelmsford    680
6           Medway    672
7           Bexley    607
8  Southend-On-Sea    586
9         Tendring    542
